<a href="https://colab.research.google.com/github/kazimianec/TARIC/blob/master/Training_TRCtransformersSBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch


In [2]:
import numpy as np
import pandas as pd
import os

In [3]:
print(torch.cuda.device_count())

1


In [4]:
url_TARIC_C2_descrptions = "https://raw.githubusercontent.com/kazimianec/TARIC/master/DATA/TARIC_C2_descriptions.csv"
url_TARIC_C4_descrptions ="https://raw.githubusercontent.com/kazimianec/TARIC/master/DATA/TARIC_C4_descriptions.csv"
url_TARIC_C6_descrptions ="https://raw.githubusercontent.com/kazimianec/TARIC/master/DATA/TARIC_C6_descriptions.csv"
url_TARIC_C246_descrptions = "https://raw.githubusercontent.com/kazimianec/TARIC/master/DATA/TARIC_C246_descriptions_CONCAT.csv"


In [5]:
import pandas as pd 
import io 
  
df_C2_descriptions = pd.read_csv(url_TARIC_C2_descrptions) 
df_C4_descriptions = pd.read_csv(url_TARIC_C4_descrptions)
df_C6_descriptions = pd.read_csv(url_TARIC_C6_descrptions)
df_C246_descriptions = pd.read_csv(url_TARIC_C246_descrptions)

In [6]:
for col in df_C246_descriptions.columns: 
    print(col)

CODE_2
CODE_4
CODE_6
goods_CODE_6
Description_46
Description_246
Description_46_concat
Description_246_concat


In [39]:
#sentences = df['CLASS_Description'].tolist()

sentences_C2 = df_C2_descriptions['Description_2'].tolist()
sentences_C4 = df_C4_descriptions['Description_4'].tolist()
sentences_C6 = df_C6_descriptions['Description_6'].tolist()

sentences_C246 = df_C246_descriptions['Description_246'].tolist()
sentences_C46 = df_C246_descriptions['Description_46'].tolist()

sentences_C246_concat = df_C246_descriptions['Description_246_concat'].tolist()
sentences_C46_concat = df_C246_descriptions['Description_46_concat'].tolist()


In [8]:
!pip install sentence_transformers

     |████████████████████████████████| 71kB 8.6MB/s 
     |████████████████████████████████| 1.8MB 24.5MB/s 
     |████████████████████████████████| 1.2MB 55.7MB/s 
     |████████████████████████████████| 890kB 53.2MB/s 
     |████████████████████████████████| 2.9MB 49.2MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.4.1.2-cp36-none-any.whl size=103068 sha256=7fd02f9fe93b629579f39e8428eb51a01ce7661f06da2966f4ed3af678c86a67
  Stored in directory: /root/.cache/pip/wheels/3d/33/d1/5703dd56199c09d4a1b41e0c07fb4e7765a84d787cbdc48ac3
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=680e259e0a035da78f0f57c492cb7a0b1b6f44d34bad3ba4aaaa2507c2fc8bf7
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [40]:
from sentence_transformers import SentenceTransformer, util


**SBERT**
Models can be loaded from SBERT , full list:
https://docs.google.com/spreadsheets/d/14QplCdTCDwEmTqrn1LH4yrbKvdogK4oQvYO1K1aPR5M/edit#gid=0


In [ ]:
#model = SentenceTransformer('average_word_embeddings_glove.840B.300d')
#model = SentenceTransformer('paraphrase-distilroberta-base-v1') --no
#model= SentenceTransformer('msmarco-distilroberta-base-v2') --no
model = SentenceTransformer('stsb-bert-large')  ##maybe
##model = SentenceTransformer('stsb-roberta-large') --no
#model= SentenceTransformer('paraphrase-xlm-r-multilingual-v1')
#model= SentenceTransformer('LaBSE')  # interesting
#model = SentenceTransformer('quora-distilbert-multilingual') ## not ok

model = SentenceTransformer('average_word_embeddings_glove.840B.300d')

100%|██████████| 1.24G/1.24G [00:40<00:00, 30.8MB/s]


In [51]:
sentences = sentences_C246_concat
sentences = [element.lower() for element in sentences]


In [52]:

embeddings = model.encode(sentences, convert_to_tensor=True)
#embeddings_246_concat = model.encode(sentences_246_concat, convert_to_tensor=True)
#embeddings_46_concat = model.encode(sentences_46_concat, convert_to_tensor=True)
#embeddings_246 = model.encode(sentences_246, convert_to_tensor=True)
#embeddings_46 = model.encode(sentences_46, convert_to_tensor=True)

In [53]:
TAROC_cosine_scores = util.pytorch_cos_sim(embeddings, embeddings)
print(TAROC_cosine_scores.shape)

top_k=5
top_results = torch.topk(TAROC_cosine_scores, k=top_k)
print(TAROC_cosine_scores.shape)
print(TAROC_cosine_scores)
print(top_results.values)
print(top_results.indices)
#print(sentences[0])
#print(sentences[40])
#y = torch.ones(TAROC_cosine_scores.shape)*0.8
#torch.where(TAROC_cosine_scores > y, TAROC_cosine_scores, 0)

torch.Size([5209, 5209])
torch.Size([5209, 5209])
tensor([[1.0000, 0.9699, 0.9475,  ..., 0.3461, 0.3495, 0.3615],
        [0.9699, 1.0000, 0.9855,  ..., 0.3131, 0.3187, 0.3271],
        [0.9475, 0.9855, 1.0000,  ..., 0.3039, 0.3085, 0.3140],
        ...,
        [0.3461, 0.3131, 0.3039,  ..., 1.0000, 0.9939, 0.9732],
        [0.3495, 0.3187, 0.3085,  ..., 0.9939, 1.0000, 0.9801],
        [0.3615, 0.3271, 0.3140,  ..., 0.9732, 0.9801, 1.0000]])
tensor([[1.0000, 0.9699, 0.9699, 0.9475, 0.8995],
        [1.0000, 1.0000, 0.9855, 0.9699, 0.8414],
        [1.0000, 0.9855, 0.9855, 0.9475, 0.8051],
        ...,
        [1.0000, 0.9939, 0.9732, 0.8756, 0.8649],
        [1.0000, 0.9939, 0.9801, 0.8694, 0.8610],
        [1.0000, 0.9801, 0.9732, 0.8708, 0.8477]])
tensor([[   0,    3,    1,    2,    4],
        [   1,    3,    2,    0,   13],
        [   2,    1,    3,    0,   13],
        ...,
        [5206, 5207, 5208, 5203, 5204],
        [5207, 5206, 5208, 5204, 5203],
        [5208, 5207, 5206

In [54]:
description = ["Vyriškos kelnės"]

description = ["Мужские джинсы"]
description = ["Beef fillet"]

description = ["Jautienos filė"]

description = ["Лак для маникюра"]

description = ["Nail polish"]

description = ["Кресла офисные"]

description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"]

description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"]
description = ["I Love Makeup - Nail Polish"]
description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"]



description = ["папиросы"]
description = ["I Love Makeup  - Nail Polish"]
description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"]
description = ["Beef fillet"]
description = ["Гoвядина филе морож"]
description = ["Мужские джинсы"]
description = ["Фанера"]
description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"]
description = ["Vineer on kihiline materjal, mis valmistatakse õhukeste puitlehtede – spoonide – kokkuliimimise teel."] 
description = ["офисные стулья деревянные дубовые"] 
description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"]
description = ["Fosmon Apple iPhone 4 / iPhone 4G USB Sync Charge Data Cable with USB Auto Car Charger and USB Home Travel Charger"] #SUPER matches S3
description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"] # not
description = ["HTC HD7 Crowned Heart Phone Protector Cover Case"] ## bullshit is both cases - bert and S3
description = ["Jolly Jumper Pashmama Nursing Cover - Tan"]
description = ["HTC HD7 Crowned Heart Phone Protector Cover Case"]
description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"]
description = ["I Love Makeup  - Nail Polish"]
description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"]
description = ["HTC HD7 Crowned Heart Phone Protector Cover Case"]


description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"]
description = ["Beef fillet"]
description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"]
description = ["Phone Protector Cover Case"]
description = ["I Love Makeup  - Nail Polish"]

embeddings_description = model.encode(description, convert_to_tensor=True )

In [55]:
cosine_scores = util.pytorch_cos_sim(embeddings, embeddings_description)
cosine_scores.shape
values,indices = torch.transpose(cosine_scores,0,1).topk(10)
print(values, indices)
indices.numpy()[0][0]

print(values.numpy()[0][0], sentences[indices.numpy()[0][0]])
print(values.numpy()[0][1], sentences[indices.numpy()[0][1]])
print(values.numpy()[0][2], sentences[indices.numpy()[0][2]])
print(values.numpy()[0][3], sentences[indices.numpy()[0][3]])
print(values.numpy()[0][4], sentences[indices.numpy()[0][4]])




tensor([[0.3525, 0.3450, 0.3440, 0.3237, 0.3102, 0.2910, 0.2849, 0.2812, 0.2770,
         0.2717]]) tensor([[1680, 1681, 1682, 1684, 1683, 3160, 1687,  354, 3161, 3158]])
0.35253096 essential oils and resinoids; perfumery, cosmetic or toilet preparations; beauty or make-up preparations and preparations for the care of the skin (other than medicaments), including sunscreen or suntan preparations; manicure or pedicure preparations; lip make-up preparations 
0.34498084 essential oils and resinoids; perfumery, cosmetic or toilet preparations; beauty or make-up preparations and preparations for the care of the skin (other than medicaments), including sunscreen or suntan preparations; manicure or pedicure preparations; eye make-up preparations 
0.34403914 essential oils and resinoids; perfumery, cosmetic or toilet preparations; beauty or make-up preparations and preparations for the care of the skin (other than medicaments), including sunscreen or suntan preparations; manicure or pedicure pr

In [ ]:
df_C246_descriptions.iloc[indices.numpy()[0],:]

**TRAINING**

In [56]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

#Define the model. Either from scratch of by loading a pre-trained model
t_model = model

#Define your train examples. You need more than just two examples...
#train_examples = [
#                  InputExample(texts=['Nail polish makeup', 'manicure or pedicure preparations'], label=1.),
#                  InputExample(texts=['Nail polish makeup', 'essential oils and resinoids; perfumery, cosmetic or toilet preparations'], label=1.),
#                  InputExample(texts=['Nail polish makeup', 'soap, organic surface-active agents, washing preparations, lubricating preparations, artificial waxes, prepared waxes, polishing or scouring preparations, candles and similar articles, modelling pastes'], label=0.)]

train_examples = [
                  InputExample(texts=['Nail polish makeup', 'manicure or pedicure preparations'], label=1.),
                  #InputExample(texts=['Nail polish makeup', 'essential oils and resinoids; perfumery, cosmetic or toilet preparations'], label=1.),
                  InputExample(texts=['Nail polish makeup', 'lips'], label=0.),
                  InputExample(texts=['Nail polish makeup', 'eye'], label=0.)]



#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=1)
train_loss = losses.CosineSimilarityLoss(model)

#Tune the model
t_model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)

RuntimeError: ignored

In [48]:
t_embeddings = t_model.encode(sentences, convert_to_tensor=True)

In [49]:
t_embeddings_description = t_model.encode(description, convert_to_tensor=True )

cosine_scores = util.pytorch_cos_sim(t_embeddings, t_embeddings_description)
cosine_scores.shape
values,indices = torch.transpose(cosine_scores,0,1).topk(10)
print(values, indices)
indices.numpy()[0][0]

print(values.numpy()[0][0], sentences[indices.numpy()[0][0]])
print(values.numpy()[0][1], sentences[indices.numpy()[0][1]])
print(values.numpy()[0][2], sentences[indices.numpy()[0][2]])
print(values.numpy()[0][3], sentences[indices.numpy()[0][3]])
print(values.numpy()[0][4], sentences[indices.numpy()[0][4]])



tensor([[0.2858, 0.2828, 0.2706, 0.2684, 0.2581, 0.2457, 0.2394, 0.2390, 0.2290,
         0.2270]]) tensor([[1680, 1681, 1682, 1683, 1684, 5071, 5069, 1704, 5070, 3830]])
0.28584677 essential oils and resinoids; perfumery, cosmetic or toilet preparations; beauty or make-up preparations and preparations for the care of the skin (other than medicaments), including sunscreen or suntan preparations; manicure or pedicure preparations; lip make-up preparations 
0.28284472 essential oils and resinoids; perfumery, cosmetic or toilet preparations; beauty or make-up preparations and preparations for the care of the skin (other than medicaments), including sunscreen or suntan preparations; manicure or pedicure preparations; eye make-up preparations 
0.2706212 essential oils and resinoids; perfumery, cosmetic or toilet preparations; beauty or make-up preparations and preparations for the care of the skin (other than medicaments), including sunscreen or suntan preparations; manicure or pedicure pre